# 4.	Time Series Analysis for Prediction
The data extracted to perform the modeling was for the period following the New UOgrade launched in December 2019. The selection of the said period was due to the impact of the New Upgrade. The efficiency of the New Upgrade had improved the number of issues logged and resolution time, as portrayed in Section 3. As a result, modeling done before the new upgrade was launched will be inaccurate.
However, the limitation to the selected period was a short duration of 17 months, where it may not contain sufficient data to develop a robust model. Thus, we have split our dataset into two parts to validate the model; a training dataset extracted from the period December 2019 to January 2021 (14 months) and test dataset from February to April 2021 (3 months). The objective is on the prediction for the following 3 months.
<br><br>
## 4.2.	Time Series Modeling – All Countries
On our dataset, multiple time series modeling was performed in order to attain the best fit time series model for prediction. The majority of the model did not perform well in terms of issue prediction. The complete list may be found in Appendix 1 and Appendix 2, where the graphs in Appendix 2 illustrate how the predicted values were unable to fit our test values.


<br><br>
### 4.2.1.	Exponential Smoothing – Holt’s
We ran Holt’s Smoothing model on a monthly basis for the training dataset, with a prediction to fit the test dataset. The graph illustrates the prediction line in blue, while the actual test dataset was in red. The predicted value and trend of the slope are different compared to our test dataset. Hence, Holt’s smoothing method is not suited for our dataset.

In [ ]:
```{r Time Series - All Country - Exponential Smoothing - Holts}
# Train the time series model by using exponential smoothing - Holt's and Holt's Winter method

# Data by month:
# plot the data to check the trend and seasonality
total.issue=month.2019$total
plot(total.issue,type = "o")
# there is trend component as shown in the graph 

<img src="https://drive.google.com/uc?export=view&id=143_vKv6sqspnPT-5QYHU8JvULoftGxni" width="400" />

In [ ]:
# Create model by [Exponential Smoothing - Holt's] method
# Run for train set
holts.training1 <- holt(holts.training,h=3)
autoplot(forecast(holts.training1))
summary(holts.training1) 

<img src="https://drive.google.com/uc?export=view&id=144ybsh20hbuzjpBQW5MmFl6JyUBV4Fdh" width="400" />
<img src="https://drive.google.com/uc?export=view&id=1476YgQwRLaU_MPeELW5s9pJbL4Z7vYY4" width="400" />


In [ ]:
# Validate the train set with test set 
holts.training1 %>% forecast(h = 3) %>% autoplot() +
  autolayer(holts.test)             
# From the graph, the predicted value from train set is not fit to test set. 

<img src="https://drive.google.com/uc?export=view&id=147r9O7l9kT6BhYS3brPWA1_5gTMO0Tdu" width="400" />

In [ ]:
# Conclude: Model from exponential smoothing by month is weak.  
# Reason: Predicted value is not fit to test set.
```

<br><br>
### 4.2.2.	Exponential Smoothing – Holt’s Winter
The Holt’s Winter Model is being used for data whereby there are trends or seasonality. Considering that we lacked 24 months of data to train the model, we failed to proceed with the modeling and encountered an error whereby the seasonal component could not be estimated in the fitting process.

In [ ]:
```{r Time Series - All Country - Exponential Smoothing - Holts Winter}
# Create model by [Exponential Smoothing - Holt's Winter] method
holts.trainingw <- subset(holts.training, start=length(holts.training)-23)
hwtstotal.issuehw <- hw(holts.trainingw,h=13)
````

<img src="https://drive.google.com/uc?export=view&id=14FuOf8kgHqwhXFlTWLMtEOnPz3ODyLJI" width="650" />

<br><br>
### 4.2.3 Stationarity check for ARIMA and SARIMA
The Autoregressive Integrated Moving Average (ARIMA) model and Seasonal Autoregressive Integrated Moving Average (SARIMA)  will be used as the other model for prediction. The series requires first differencing to attain stationarity, according to the Augmented Dickey-Fuller (ADF) Test and the Kwiatkowski-Phillips-Schmidt-Shin (KPSS) Test.

In [ ]:
```{r Time Series - All Country - ARIMA by month}
# Train the time series model by using ARIMA with data by month

# Check the [Stationarity] of time series data
# Augmented Dickey-Fuller Test (ADF)
adf.test(tstotal.issue)

<img src="https://drive.google.com/uc?export=view&id=14Jcd1qqdgSM7Tld0sjl1L27nMN-cUMWs" width="400" />

- HO: Series is Non-Stationary; H1: Series is Stationary
- P-value > 0.05, failed to reject HO; series is non-stationary.


In [ ]:
# Kwiatkowski-Phillips-Schmidt-Shin (KPSS)
kpss.test(tstotal.issue, null="Trend")
```

<img src="https://drive.google.com/uc?export=view&id=14KpQCC2W4U7vcUhjCoJZFRFwQe2G_EEa" width="520" />

 - HO: Series is Stationary; H1: Series is Non-Stationary
 - P-value > 0.05, failed to reject HO; series is stationary

As one of the test has proved the data is non-stationary, differencing is required to be done while training the ARIMA and SARIMA model. 

<br><br>
#### 4.2.4 ARIMA
The ARIMA model was trained by using monthly, weekly and daily dataset, as well as different parameters selected from the ACF and PACF plot, the comparison of models may be found from Appendix 1 and Appendix 2. However, all ARIMA models are weak, the predicted value from train set is not fit to the test set. Below is the illustration from the one of the better ARIMA model:

In [ ]:
```{r Time Series - All Country - ARIMA by day}
# Train the time series model by using ARIMA with data by day
# As checked from section above, the dataset is non-stationery, differencing is required
# 1st differencing, check the ACF and PACF plot
d.training %>% diff()%>% ggtsdisplay()

*First differencing:*

<img src="https://drive.google.com/uc?export=view&id=14LETqBwyn8-ppvKCMuZJ9rS5mnG9jKuN" width="400" />



In [ ]:
# Arima on train set (6,1,0)
dmodel.train = arima(d.training, order=c(6,1,0))
summary(dmodel.train)
checkresiduals(dmodel.train) 

<img src="https://drive.google.com/uc?export=view&id=14Rk_ys-fdzkaaSot7aMOzDzPADKElRtz" width="500" /> 
<img src="https://drive.google.com/uc?export=view&id=14kb0BIynz8QDNo8-o5QEjqV6rcAm-JGX" width="450" />

In [ ]:
# Plot it in test set
dmodel.train %>% forecast(h = 87) %>% autoplot() + autolayer(d.test)
pred.test = Arima(d.test, model = dmodel.train) 
accuracy(pred.test)
# The predicted value from train set is different from test set

# Conclude: Model from ARIMA by day is weak. 
# Reason: Predicted value is not fit to test set.
```

<img src="https://drive.google.com/uc?export=view&id=14maoajOxa6s-sBdCG4Hr_N9I5EIfZYzE" width="450" /> 

<img src="https://drive.google.com/uc?export=view&id=14o0ZbhaViqQaPqRGQD7XEdmI_HP9mJ-z" width="450" />


# Appendix
*Appendix 1: Table of Comparison of Time Series Prediction Model of Monthly Issue of All Countries.*
*<img src="https://drive.google.com/uc?export=view&id=13zzrUhiEuUt8zduDf4lusbBQ2tM6AlFP" width="600" />*
<br><br>
*Appendix 2: Charts of Comparison of Time Series Prediction Model of Monthly Issue of All Countries.*
<img src="https://drive.google.com/uc?export=view&id=142lkyl5K4RlsUT6l5SuGzmEXH0x409ol" width="600" />